In [ ]:
import os
import leafmap 
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

import fiona
from shapely import Polygon
from shapely import wkb
import matplotlib.pyplot as plt
from shapely.plotting import plot_polygon

import folium

In [2]:
m = leafmap.Map(center=[44.53831 , 11.40759], zoom=15)
m.add_basemap("SATELLITE")
m

Map(center=[44.53831, 11.40759], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

In [3]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [11.3996, 44.5359, 11.4057, 44.5415]


In [4]:
# different basemaps
# get_basemaps().keys()
image = "satellite.tif"
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True)

Downloaded image 001/456
Downloaded image 002/456
Downloaded image 003/456
Downloaded image 004/456
Downloaded image 005/456
Downloaded image 006/456
Downloaded image 007/456
Downloaded image 008/456
Downloaded image 009/456
Downloaded image 010/456
Downloaded image 011/456
Downloaded image 012/456
Downloaded image 013/456
Downloaded image 014/456
Downloaded image 015/456
Downloaded image 016/456
Downloaded image 017/456
Downloaded image 018/456
Downloaded image 019/456
Downloaded image 020/456
Downloaded image 021/456
Downloaded image 022/456
Downloaded image 023/456
Downloaded image 024/456
Downloaded image 025/456
Downloaded image 026/456
Downloaded image 027/456
Downloaded image 028/456
Downloaded image 029/456
Downloaded image 030/456
Downloaded image 031/456
Downloaded image 032/456
Downloaded image 033/456
Downloaded image 034/456
Downloaded image 035/456
Downloaded image 036/456
Downloaded image 037/456
Downloaded image 038/456
Downloaded image 039/456
Downloaded image 040/456


In [ ]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    sam_kwargs=None,
)

mask = "segment.tif"
sam.generate(
    image, mask, batch=True, foreground=True, erosion_kernel=(3, 3), mask_multiplier=255
)

vector = "segment.gpkg"
sam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)

In [10]:
vector = "segment.gpkg"
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
m.add_vector(vector, layer_name="Vector", style=style)
m

Map(bottom=3033046.0, center=[44.53831, 11.40759], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [6]:
import geopandas as gpd
data = gpd.read_file('segment.gpkg')
data.to_crs('EPSG:4326', inplace=True)

data.shape
areas = []
for _, r in data.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"], crs=data.crs).simplify(tolerance=0.0001)
    center = sim_geo.centroid
    area_proj = sim_geo.to_crs("+proj=cea +lat_ts="+str(center.y)+" +lon_ts="+str(center.x)+" +units=m")
    areas.append(area_proj.at[0].area)

data['area'] = areas

data.shape
m = folium.Map(location = [44.53831 , 11.40759], zoom_start=15)

for _, r in data[data['area']>400].iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    p = list(r['geometry'].exterior.coords)
    res = [(x,y) for y,x in p]
    #print(p)
    #geo_j = Polygon(p).to_json()

    geo_j = folium.PolyLine(res, style_function=lambda x: {"fillColor": "orange"})

    geo_j.add_to(m)
    
m

In [29]:
from scipy.spatial import ConvexHull

In [48]:
m = folium.Map(location = [44.53831 , 11.40759], zoom_start=15)

p = data[data['area']>400]['geometry'].values[8].exterior.coords
res = [(x,y) for y,x in p]
vertices = ConvexHull(res, True).simplices

geo_j = folium.PolyLine(np.array(res)[vertices], style_function=lambda x: {"fillColor": "orange"})

geo_j.add_to(m)
m

## FAST SAM DA FARE FINE TUNING (ONE/FEW SHOTS)

In [ ]:
from samgeo.fast_sam import SamGeo

In [ ]:
sam = SamGeo(model="FastSAM-x.pt")
sam.set_image(image)
#sam.box_prompt(bbox)
sam.everything_prompt(output="mask.tif")


image 1/1 /Users/andreafrasson/Desktop/GetPolygons/satellite.tif: 1024x800 90 objects, 1801.9ms
Speed: 3.3ms preprocess, 1801.9ms inference, 35.0ms postprocess per image at shape (1, 3, 1024, 800)


In [7]:
sam.raster_to_vector("mask.tif", "mask.geojson")

In [8]:
m.add_raster("mask.tif", opacity=0.5, layer_name="Mask")
m.add_vector("mask.geojson", layer_name="Mask Vector")
m

Map(bottom=3033033.0, center=[44.5387, 11.40265], controls=(ZoomControl(options=['position', 'zoom_in_text', '…